<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация---очистка-тренировочного-корпуса-текстов" data-toc-modified-id="Лемматизация---очистка-тренировочного-корпуса-текстов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация - очистка тренировочного корпуса текстов</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Балансируем-выборку" data-toc-modified-id="Балансируем-выборку-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Балансируем выборку</a></span></li><li><span><a href="#СatboostClassifier" data-toc-modified-id="СatboostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>СatboostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span><ul class="toc-item"><li><span><a href="#Тест-моделей" data-toc-modified-id="Тест-моделей-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Тест моделей</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Анализ</a></span></li></ul></li></ul></div>

# Описание

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

import catboost
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import re

from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")
                                     
def data_view(data):
    
    """
    Функция вызвращает данные о таблице
    """
    return display(data.info(),
                   data.isna().sum(), 
                   data.duplicated().sum(),
                   data.head(),
                   data.tail(),
                  )

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data_view(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

text     0
toxic    0
dtype: int64

0

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


In [4]:
data['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [5]:
data['text'] = data['text'].str.lower()

- Разделим данные на обучающую и тестовую выборки. 

In [6]:
train, test = train_test_split(data, test_size=0.1, stratify=data['toxic'])

In [7]:
train['toxic'].value_counts(normalize=True), test['toxic'].value_counts(normalize=True)

(0    0.898324
 1    0.101676
 Name: toxic, dtype: float64,
 0    0.898296
 1    0.101704
 Name: toxic, dtype: float64)

In [8]:
train.shape, test.shape

((143613, 2), (15958, 2))

### Лемматизация - очистка тренировочного корпуса текстов

- Функция для очистки текстов при помощи рег.выражений

In [9]:
lemm = WordNetLemmatizer()

In [10]:
def clear_text(text):
    re_text = re.sub(r'[^\w\']', ' ', text)
    return " ".join(re_text.split())

- Пример преобразования произвольного текста

In [11]:
data['text'][3]

'"\nmore\ni can\'t make any real suggestions on improvement - i wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -i think the references may need tidying so that they are all in the exact same format ie date format etc. i can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nthere appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up. it\'s listed in the relevant form eg wikipedia:good_article_nominations#transport  "'

In [12]:
clear_text(data['text'][3])

"more i can't make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no one else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it's listed in the relevant form eg wikipedia good_article_nominations transport"

- Функция лемматизирует слова с учётом частей речи

In [13]:
def lemm_pos(text):
    
    text_list=[]
    
    for word in nltk.word_tokenize(clear_text(text)):
        try:
            word = lemm.lemmatize(word, pos=nltk.pos_tag(
                                    [word])[0][1][0].lower())
            text_list.append(word)
        except:
            text_list.append(word)

    return " ".join(text_list)

In [14]:
stopwords = set(nltk_stopwords.words('english'))

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [16]:
# train['lemm_text'] = train['text'].apply(lambda x: lemm_pos(x))

# corpus_train = train['lemm_text'].values.astype('U')
# tf_idf_train = count_tf_idf.fit_transform(corpus_train)

In [17]:
# tf_idf_train.shape

### Вывод

- Тексты комментариев имеют много лишних символов.

- Выборка несбалансированная, 90% - нетоксичных комментариев, только 10% - токсичных.

- Была проведена очистка текста с помощью рег. выражений и лемматизация с учётом части речи где это возможно определить.

## Обучение

### LogisticRegression

In [18]:
# model = LogisticRegression(random_state=12345, class_weight='balanced', C=0.8)

# cross_val_score(model, tf_idf_train, train['toxic'], scoring='f1', cv=3) 

### СatboostClassifier

In [19]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [20]:
train_cat, test_cat = train_test_split(data, test_size=0.1, stratify=data['toxic'])

In [21]:
train_cat['toxic'].value_counts()

0    129011
1     14602
Name: toxic, dtype: int64

In [22]:
train_cat['text'] = train_cat['text'].apply(lambda x: clear_text(x))
test_cat['text'] = test_cat['text'].apply(lambda x: clear_text(x))

In [25]:
model_cat = CatBoostClassifier(eval_metric='F1',
                           text_features=['text'],
                           tokenizers=[{
                               'tokenizer_id': 'Space',
                               'delimiter': ' ',
                               'separator_type': 'ByDelimiter',
                           },{
                               'tokenizer_id': 'Sense',
                               'separator_type': 'BySense',
                           }],
                           dictionaries = [{
                              'dictionary_id': 'Word',
                              'max_dictionary_size': '50000',
                              'gram_count': '1',
                           },{
                              'dictionary_id': 'BiGram',
                              'max_dictionary_size': '50000',
                              'gram_count': '2',
                           }],
                           
                           feature_calcers=['NaiveBayes', 
                                            'BoW'],
                           
                           random_state=12345)

model_cat.fit(train_cat[['text']], train_cat['toxic'])

Learning rate set to 0.085912
0:	learn: 0.6535859	total: 1.09s	remaining: 18m 11s
1:	learn: 0.6757409	total: 2.3s	remaining: 19m 6s
2:	learn: 0.6754762	total: 3.4s	remaining: 18m 48s
3:	learn: 0.6758069	total: 4.52s	remaining: 18m 45s
4:	learn: 0.6750839	total: 5.7s	remaining: 18m 54s
5:	learn: 0.6761890	total: 6.76s	remaining: 18m 39s
6:	learn: 0.6773052	total: 7.84s	remaining: 18m 32s
7:	learn: 0.6796620	total: 8.89s	remaining: 18m 22s
8:	learn: 0.6814360	total: 10s	remaining: 18m 23s
9:	learn: 0.6787310	total: 11.1s	remaining: 18m 14s
10:	learn: 0.6800290	total: 12.4s	remaining: 18m 34s
11:	learn: 0.6818597	total: 13.5s	remaining: 18m 31s
12:	learn: 0.6835232	total: 14.8s	remaining: 18m 40s
13:	learn: 0.6850686	total: 16s	remaining: 18m 49s
14:	learn: 0.6852895	total: 17.2s	remaining: 18m 50s
15:	learn: 0.6843390	total: 18.3s	remaining: 18m 43s
16:	learn: 0.6857276	total: 19.5s	remaining: 18m 47s
17:	learn: 0.6872754	total: 20.6s	remaining: 18m 43s
18:	learn: 0.6892194	total: 21.8s	

In [26]:
pred = model_cat.predict(train_cat[['text']])

f1_score(train_cat['toxic'], pred)

0.8334981866139136

## Выводы

### Тест моделей

- LogisticRegression

In [30]:
test['lemm_text'] = test['text'].apply(lambda x: lemm_pos(x))
corpus_test = test['lemm_text'].values.astype('U')
tf_idf_test = count_tf_idf.transform(corpus_test)

In [31]:
model_log.fit(tf_idf_train, train['toxic'])
pred = model_log.predict(tf_idf_test)

f1_score(test['toxic'], pred)

0.6828798458945341

- CatBoostClassifier

In [32]:
test_cat['text'] = test_cat['text'].apply(lambda x: clear_text(x))

In [33]:
pred = model_cat.predict(test_cat[['text']])

f1_score(test_cat['toxic'], pred)

0.7778528021607022

### Анализ

- Были проведены испытания 2х моделей. `LogisticRegression` и `CatBoostClassifier`.


- `LogisticRegression` показала качество 0.75 на несбаласированных данных и 0.88 на сбалансированных данных на тренировочной выборке и 0.68 на тестовой. Для этой модели был выбран алгоритм векторизации текста TF-IDF.


- Для `CatBoostClassifier` был использован внутренний алгоритм токенезации, лемматизации и векторизации текста. Модель показала 0.83 на тестовой выборке и 0.77 на тестовой.